In [11]:
!pip install transformers -q

In [27]:
from transformers import pipeline, AutoTokenizer, AutoModel, AutoModelForSequenceClassification

import torch

In [28]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [29]:
raw_inputs = [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]

inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors='pt')

In [35]:
outputs = model(**inputs)
with torch.inference_mode():
  fin = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(fin)

tensor([[4.0195e-02, 9.5980e-01],
        [9.9946e-01, 5.4418e-04]])


In [36]:
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
channels = 3000
samples = 20

weight = torch.ones((1, channels, 1, 1))
image = torch.rand((samples, channels, 10, 10))

noise = torch.randn((samples, 1, 10, 10))
temp = weight * noise
image_noisy = image + temp
print(image.shape, temp.shape, image_noisy.shape)

torch.Size([20, 3000, 10, 10]) torch.Size([20, 3000, 10, 10]) torch.Size([20, 3000, 10, 10])


In [ ]:
print(torch.abs(image - image_noisy).std(0).mean())
print(torch.abs(image - image_noisy).std(1).mean())
print(torch.abs(image - image_noisy).std(2).mean())
print(torch.abs(image - image_noisy).std(3).mean())

tensor(0.5843)
tensor(2.4045e-08)
tensor(0.5671)
tensor(0.5714)
